# Chapter 3. 제약 충족 문제
- 제약 충족 문제는 도메인이라는 범위에 속하는 값을 갖는 변수로 구성
- 변수 사이의 제약 조건을 충족하여 문제 해결

제약 충족 문제의 핵심 개념
- 제약 조건 변수(variables)
- 도메인
- 제약 조건(Constraint class) 
    - (충족 확인 메서드: satisfied())

해결 방법
- Prolog, Picat 같은 프로그래밍 언어는 제약 충족 문제를 해결할 수 있는 함수 제공
- 일반적으로는 백트래킹 검색과 이를 향상시키는 몇가지 휴리스틱을 통합하여 프레임워크 구축

이 장에서 살펴볼 것
- 재귀 백트래킹 검색을 사용한 프레임워크

## 3.1 제약 충족 문제 프레잌워크 구현하기
제약 조건 Constraint 클래스로 정의한다. 이 클래스는 제약 조건 변수(variables)와 이를 충족하는지 검사하는 메서드(satisfied())로 구성된다.

Constraint 클래스를 추상 클래스로 정의하여 기본 구현을 오버라이드

제약 조건이 충족되었는지 판단하는 것이 핵심

In [1]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # Variable type
D = TypeVar('D') # Domail type

# base class for Constraint
class Constraint(Generic[V, D], ABC):
    # variables
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        pass


추상 클래스
- 클래스 계층 구조에서 탬플릿 역할

CSP 클래스
- 변수, 도메인, 제약조건 저장
- 타입 힌트에서 제네릭을 사용하여 V, D 값을 유연하게 처리
- 변수 컬렉션: 변수의 리스트
- 도메인 컬렉션: 변수에 가능한 값 리스트를 매핑하는 딕셔너리
- 제약조건 컬렉션: 각 변수에 제약조건(Constraint 클래스) 리스트로 매핑된 딕셔너리

In [2]:
from typing import Generic, TypeVar, Dict, List, Optional

class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # 제약조건을 확인할 변수
        self.domains: Dict[V, List[Constraint[V, D]]] = {} # 각 변수의 도메인
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("모든 변수에 도메인이 할당되어야 합니다.")
            
    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("제약 조건 변수가 아닙니다.")
            else:
                self.constraint[variable].append(constraint)
    
    # 주어진 변수의 모든 제약 조건을 검사하여 assignment 값이 일관적인지 확인한다.
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

In [ ]:
# 주어진 변수의 모ㄴ 제ㄱ 조ㄴㅡㄹ 검ㅏㅏ여 assignment 값이 일관적인지 확인
def consistent(self, variable:V, assignment: Dict[V, D]) -> bool:
    for constraint in self.constraints[variable]:
        if not constraint.satisfied(assignment):
            return False
    return True

def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
    # assignment는 모든 변수가 할당될 때 완료된다.(기저조건)
    if len(assignment) == len(self.variables):
        return assignment
    
    # 할당되지 않은 모든 변수를 가져온다
    unassigned: List[V] = [v for v in self.variables if v not in assignment]

    # 할당되지 않은 첫 번째 변수의 가능한 모든 도메인 값을 가져온다.
    first: V = unassigned[0]
    for value in self.domains[first]:
        # 해당 변수에 가능한 모든 도메인 값 할당
        local_assignment = assignment.copy()
        local_assignment[first] = value
        #local_assignment 값이 일관적이면 재귀 호출
        # 모든 제약 조건과 일치하면(consistent() 메소드 True) 새 할당을 제자리에서(in place) 검색
        if self.consistent(first, local_assignment):
            result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
            # 결과를 찾지 못하면 백트래킹을 종료
            if result is not None:
                return result
    return None # 솔루션 없음 # backtracking

## 3.2 호주 지도 색칠 문제
- 변수: 호주 7개 지역
- 도메인: 색깔(빨, 파, 녹)
- 제약 조건: 인접한 두 지역은 같은 색으로 칠할 수 없다(이진(binary) 제약 조건)

In [ ]:
from typing import Dict, List, Optional

class MapColoringConstraint(Constraint[str, str]):
    def __init__(self, place1: str, place2: str) -> None:
        super.__init__([place1, place2])
        self.place1: str = place1
        self.place2: str = place2

    def satisfied(self, assignment: Dict[str, str]) -> bool:
        if self.place1 not in assignment or self.place2 not in assignment:
            return True
        return assignment[self.place1] != assignment[self.place2]

In [ ]:
class QueenConstraint(Constraint[int, int]):
    def __init__(self, columns: List[int]) -> None:
        super().__init__(columns)
        self.columns: List[int] = columns
